In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# config.py

In [2]:
# -*- coding: utf-8 -*-
#import tensorflow as tf

tf.app.flags.DEFINE_string('f', '', 'kernel') # 주피터에서 커널에 전달하기 위한 프레그 방법 #check???
tf.app.flags.DEFINE_integer('batch_size', 64, 'batch size')  # 배치 크기
tf.app.flags.DEFINE_integer('train_steps', 20000, 'train steps')  # 학습 에포크
tf.app.flags.DEFINE_float('dropout_width', 0.5, 'dropout width')  # 드롭아웃 크기
tf.app.flags.DEFINE_integer('embedding_size', 128, 'embedding size')  # 가중치 크기 # 논문 512 사용
tf.app.flags.DEFINE_float('learning_rate', 1e-3, 'learning rate')  # 학습률
tf.app.flags.DEFINE_integer('shuffle_seek', 1000, 'shuffle random seek')  # 셔플 시드값
tf.app.flags.DEFINE_integer('max_sequence_length', 25, 'max sequence length')  # 시퀀스 길이
tf.app.flags.DEFINE_integer('model_hidden_size', 128, 'model weights size')  # 모델 가중치 크기
tf.app.flags.DEFINE_integer('ffn_hidden_size', 512, 'ffn weights size')  # ffn 가중치 크기
tf.app.flags.DEFINE_integer('attention_head_size', 4, 'attn head size')  # 멀티 헤드 크기
tf.app.flags.DEFINE_integer('layer_size', 2, 'layer size')  # 논문은 6개 레이어이나 2개 사용 학습 속도 및 성능 튜닝
tf.app.flags.DEFINE_string('data_path', '../data_in/ChatBotData.csv', 'data path')  # 데이터 위치
tf.app.flags.DEFINE_string('vocabulary_path', './data_out/vocabularyData.voc', 'vocabulary path')  # 사전 위치
tf.app.flags.DEFINE_string('check_point_path', './data_out/check_point', 'check point path')  # 체크 포인트 위치
tf.app.flags.DEFINE_boolean('tokenize_as_morph', False, 'set morph tokenize')  # 형태소에 따른 토크나이징 사용 유무
tf.app.flags.DEFINE_boolean('xavier_initializer', True, 'set xavier initializer')  # xavier initializer를 사용할 것인지에 대한 

# Define FLAGS
DEFINES = tf.app.flags.FLAGS

In [ ]:
#if __name__ == '__main__':
#    tf.logging.set_verbosity(tf.logging.INFO)
#    tf.app.run(main)

# data.py

In [3]:
from konlpy.tag import Twitter
import pandas as pd
#import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split
import numpy as np
#from configs import DEFINES

from tqdm import tqdm

In [17]:
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

In [18]:
def load_data():
    # 판다스를 통해서 데이터를 불러온다.
    data_df = pd.read_csv(DEFINES.data_path, header=0)
    # 질문과 답변 열을 가져와 question과 answer에 넣는다.
    question, answer = list(data_df['Q']), list(data_df['A'])
    # skleran에서 지원하는 함수를 통해서 학습 셋과 
    # 테스트 셋을 나눈다.
    train_input, eval_input, train_label, eval_label = train_test_split(question, answer, test_size=0.33,
                                                                        random_state=42)
    # 그 값을 리턴한다.
    return train_input, train_label, eval_input, eval_label

In [19]:
def prepro_like_morphlized(data):
    # 형태소 분석 모듈 객체를
    # 생성합니다.

    morph_analyzer = Twitter()
    # 형태소 토크나이즈 결과 문장을 받을
    #  리스트를 생성합니다.
    result_data = list()
    # 데이터에 있는 매 문장에 대해 토크나이즈를
    # 할 수 있도록 반복문을 선언합니다.
    for seq in tqdm(data):
        # Twitter.morphs 함수를 통해 토크나이즈 된
        # 리스트 객체를 받고 다시 공백문자를 기준으로
        # 하여 문자열로 재구성 해줍니다.
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)

    return result_data

In [20]:
# 인덱스화 할 value와 키가 워드이고 
# 값이 인덱스인 딕셔너리를 받는다.
def enc_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의 
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때 
        # 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로 
        # 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고 
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는 
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 
    # 그 길이를 넘겨준다.  
    return np.asarray(sequences_input_index), sequences_length

In [21]:
# 인덱스화 할 value 키가 워드 이고 값이 
# 인덱스인 딕셔너리를 받는다.
def dec_output_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다)
    sequences_output_index = []
    # 하나의 디코딩 입력 되는 문장의 
    # 길이를 가지고 있다.(누적된다)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 디코딩 할때 가지고 
        # 있기 위한 배열이다.
        sequence_index = []
        # 디코딩 입력의 처음에는 START가 와야 하므로 
        # 그 값을 넣어 주고 시작한다.
        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의 
        # 값인 인덱스를 넣어 준다.
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_output_index 넣어 준다.
        sequences_output_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_output_index), sequences_length

In [22]:
# 인덱스화 할 value와 키가 워드 이고
# 값이 인덱스인 딕셔너리를 받는다.
def dec_target_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는 
    # 배열이다.(누적된다)
    sequences_target_index = []
    # 형태소 토크나이징 사용 유무
    if DEFINES.tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는 
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서 
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= DEFINES.max_sequence_length:
            sequence_index = sequence_index[:DEFINES.max_sequence_length - 1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가 
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (DEFINES.max_sequence_length - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을 
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다. 
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)

In [23]:
# 인덱스를 스트링으로 변경하는 함수이다.
# 바꾸고자 하는 인덱스 value와 인덱스를 
# 키로 가지고 있고 값으로 단어를 가지고 있는 
# 딕셔너리를 받는다.
def pred2string(value, dictionary):
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    print(value)
    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    for v in value:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        print(v['indexs'])
        for index in v['indexs']:
            print(index)
        sentence_string = [dictionary[index] for index in v['indexs']]

    print("***********************")
    print(sentence_string)
    print("***********************")
    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word not in PAD and word not in END:
            answer += word
            answer += " "
    # 결과를 출력한다.
    print(answer)
    return answer

In [24]:
def pred_next_string(value, dictionary):
    # 텍스트 문장을 보관할 배열을 선언한다.
    sentence_string = []
    is_finished = False

    # 인덱스 배열 하나를 꺼내서 v에 넘겨준다.
    for v in value:
        # 딕셔너리에 있는 단어로 변경해서 배열에 담는다.
        sentence_string = [dictionary[index] for index in v['indexs']]

    answer = ""
    # 패딩값도 담겨 있으므로 패딩은 모두 스페이스 처리 한다.
    for word in sentence_string:
        if word == END:
            is_finished = True
            break

        if word != PAD and word != END:
            answer += word
            answer += " "

    # 결과를 출력한다.
    return answer, is_finished

In [25]:
def rearrange(input, output, target):
    features = {"input": input, "output": output}
    return features, target

In [26]:
# 학습에 들어가 배치 데이터를 만드는 함수이다.
def train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # train_input_enc, train_output_dec, train_target_dec 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((train_input_enc, train_output_dec, train_target_dec))
    # 전체 데이터를 썩는다.
    dataset = dataset.shuffle(buffer_size=len(train_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "train batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    dataset = dataset.repeat()
    # make_one_shot_iterator를 통해 이터레이터를 
    # 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 텐서 
    # 개체를 넘겨준다.
    return iterator.get_next()

In [27]:
# 평가에 들어가 배치 데이터를 만드는 함수이다.
def eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, batch_size):
    # Dataset을 생성하는 부분으로써 from_tensor_slices부분은 
    # 각각 한 문장으로 자른다고 보면 된다.
    # eval_input_enc, eval_output_dec, eval_target_dec 
    # 3개를 각각 한문장으로 나눈다.
    dataset = tf.data.Dataset.from_tensor_slices((eval_input_enc, eval_output_dec, eval_target_dec))
    # 전체 데이터를 섞는다.
    dataset = dataset.shuffle(buffer_size=len(eval_input_enc))
    # 배치 인자 값이 없다면  에러를 발생 시킨다.
    assert batch_size is not None, "eval batchSize must not be None"
    # from_tensor_slices를 통해 나눈것을 
    # 배치크기 만큼 묶어 준다.
    dataset = dataset.batch(batch_size, drop_remainder=True)
    # 데이터 각 요소에 대해서 rearrange 함수를 
    # 통해서 요소를 변환하여 맵으로 구성한다.
    dataset = dataset.map(rearrange)
    # repeat()함수에 원하는 에포크 수를 넣을수 있으면 
    # 아무 인자도 없다면 무한으로 이터레이터 된다.
    # 평가이므로 1회만 동작 시킨다.
    dataset = dataset.repeat(1)
    # make_one_shot_iterator를 통해 
    # 이터레이터를 만들어 준다.
    iterator = dataset.make_one_shot_iterator()
    # 이터레이터를 통해 다음 항목의 
    # 텐서 개체를 넘겨준다.
    return iterator.get_next()

In [28]:
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을 
        # 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진 
    # 값들을 넘겨 준다.
    return [word for word in words if word]

In [29]:
def load_vocabulary():
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장 진행한다. 
    # 그 파일의 존재 유무를 확인한다.
    if (not (os.path.exists(DEFINES.vocabulary_path))):
        # 이미 생성된 사전 파일이 존재하지 않으므로 
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서 
        # 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(DEFINES.data_path)):
            # 데이터가 존재하니 판단스를 통해서 
            # 데이터를 불러오자
            data_df = pd.read_csv(DEFINES.data_path, encoding='utf-8')
            # 판다스의 데이터 프레임을 통해서 
            # 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])
            if DEFINES.tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
                question = prepro_like_morphlized(question)
                answer = prepro_like_morphlized(answer)
            data = []
            # 질문과 답변을 extend을 
            # 통해서 구조가 없는 배열로 만든다.
            data.extend(question)
            data.extend(answer)
            # 토큰나이져 처리 하는 부분이다.
            words = data_tokenizer(data)
            # 공통적인 단어에 대해서는 모두 
            # 필요 없으므로 한개로 만들어 주기 위해서
            # set해주고 이것들을 리스트로 만들어 준다.
            words = list(set(words))
            # 데이터 없는 내용중에 MARKER를 사전에 
            # 추가 하기 위해서 아래와 같이 처리 한다.
            # 아래는 MARKER 값이며 리스트의 첫번째 부터 
            # 순서대로 넣기 위해서 인덱스 0에 추가한다.
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNWON>"     
            words[:0] = MARKER
        # 사전 리스트를 사전 파일로 만들어 넣는다.
        with open(DEFINES.vocabulary_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')

    # 사전 파일이 존재하면 여기에서 
    # 그 파일을 불러서 배열에 넣어 준다.
    with open(DEFINES.vocabulary_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # 배열에 내용을 키와 값이 있는 
    # 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)
    # 두가지 형태의 키와 값이 있는 형태를 리턴한다. 
    # (예) 단어: 인덱스 , 인덱스: 단어)
    return char2idx, idx2char, len(char2idx)

In [30]:
def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인 
    # 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인 
    # 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.
    return char2idx, idx2char

In [32]:
def main_data(self):
    DATA_OUT_PATH = './data_out/'
    data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)
    os.makedirs(data_out_path, exist_ok=True)
    char2idx, idx2char, vocabulary_length = load_vocabulary()

# model.py

In [33]:
# -*- coding: utf-8 -*-
#import tensorflow as tf
import sys

#from configs import DEFINES
import numpy as np


def layer_norm(inputs, eps=1e-6):
    # LayerNorm(x + Sublayer(x))
    feature_shape = inputs.get_shape()[-1:]
    #  평균과 표준편차을 넘겨 준다.
    mean = tf.keras.backend.mean(inputs, [-1], keepdims=True)
    std = tf.keras.backend.std(inputs, [-1], keepdims=True)
    beta = tf.Variable(tf.zeros(feature_shape), trainable=False)
    gamma = tf.Variable(tf.ones(feature_shape), trainable=False)

    return gamma * (inputs - mean) / (std + eps) + beta


def sublayer_connection(inputs, sublayer, dropout=0.2):
    # LayerNorm(x + Sublayer(x))
    outputs = layer_norm(inputs + tf.keras.layers.Dropout(dropout)(sublayer))
    return outputs


def feed_forward(inputs, num_units):
    # FFN(x) = max(0, xW1 + b1)W2 + b2
    feature_shape = inputs.get_shape()[-1]
    inner_layer = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(inputs)
    outputs = tf.keras.layers.Dense(feature_shape)(inner_layer)

    return outputs


def positional_encoding(dim, sentence_length):
    # Positional Encoding
    # paper: https://arxiv.org/abs/1706.03762
    # P E(pos,2i) = sin(pos/100002i/dmodel)
    # P E(pos,2i+1) = cos(pos/100002i/dmodel)
    encoded_vec = np.array([pos / np.power(10000, 2 * i / dim)
                            for pos in range(sentence_length) for i in range(dim)])
    encoded_vec[::2] = np.sin(encoded_vec[::2])
    encoded_vec[1::2] = np.cos(encoded_vec[1::2])
    return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype=tf.float32)


def scaled_dot_product_attention(query, key, value, masked=False):
    # Attention(Q, K, V ) = softmax(QKt / root dk)V
    key_dim_size = float(key.get_shape().as_list()[-1])
    key = tf.transpose(key, perm=[0, 2, 1])
    outputs = tf.matmul(query, key) / tf.sqrt(key_dim_size)

    if masked:
        diag_vals = tf.ones_like(outputs[0, :, :])
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()
        masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1])

        paddings = tf.ones_like(masks) * (-2 ** 32 + 1)
        outputs = tf.where(tf.equal(masks, 0), paddings, outputs)

    attention_map = tf.nn.softmax(outputs)

    return tf.matmul(attention_map, value)


def multi_head_attention(query, key, value, num_units, heads, masked=False):
    query = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(query)
    key = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(key)
    value = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(value)

    query = tf.concat(tf.split(query, heads, axis=-1), axis=0)
    key = tf.concat(tf.split(key, heads, axis=-1), axis=0)
    value = tf.concat(tf.split(value, heads, axis=-1), axis=0)

    attention_map = scaled_dot_product_attention(query, key, value, masked)

    attn_outputs = tf.concat(tf.split(attention_map, heads, axis=0), axis=-1)
    attn_outputs = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(attn_outputs)

    return attn_outputs


def encoder_module(inputs, model_dim, ffn_dim, heads):
    self_attn = sublayer_connection(inputs, multi_head_attention(inputs, inputs, inputs,
                                                                 model_dim, heads))
    outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
    return outputs


def decoder_module(inputs, encoder_outputs, model_dim, ffn_dim, heads):
    masked_self_attn = sublayer_connection(inputs, multi_head_attention(inputs, inputs, inputs,
                                                                        model_dim, heads, masked=True))
    self_attn = sublayer_connection(masked_self_attn, multi_head_attention(masked_self_attn, encoder_outputs,
                                                                           encoder_outputs, model_dim, heads))
    outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))

    return outputs


def encoder(inputs, model_dim, ffn_dim, heads, num_layers):
    outputs = inputs
    for i in range(num_layers):
        outputs = encoder_module(outputs, model_dim, ffn_dim, heads)

    return outputs


def decoder(inputs, encoder_outputs, model_dim, ffn_dim, heads, num_layers):
    outputs = inputs
    for i in range(num_layers):
        outputs = decoder_module(outputs, encoder_outputs, model_dim, ffn_dim, heads)

    return outputs


def Model(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    position_encode = positional_encoding(params['embedding_size'], params['max_sequence_length'])

    if params['xavier_initializer']:
        embedding_initializer = 'glorot_normal'
    else:
        embedding_initializer = 'uniform'

    embedding = tf.keras.layers.Embedding(params['vocabulary_length'],
                                          params['embedding_size'],
                                          embeddings_initializer=embedding_initializer)

    x_embedded_matrix = embedding(features['input']) + position_encode
    y_embedded_matrix = embedding(features['output']) + position_encode

    encoder_outputs = encoder(x_embedded_matrix, params['model_hidden_size'], params['ffn_hidden_size'],
                              params['attention_head_size'], params['layer_size'])
    decoder_outputs = decoder(y_embedded_matrix, encoder_outputs, params['model_hidden_size'],
                              params['ffn_hidden_size'],
                              params['attention_head_size'], params['layer_size'])

    logits = tf.keras.layers.Dense(params['vocabulary_length'])(decoder_outputs)

    predict = tf.argmax(logits, 2)

    if PREDICT:
        predictions = {
            'indexs': predict,
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # 정답 차원 변경을 한다. [배치 * max_sequence_length * vocabulary_length]  
    # logits과 같은 차원을 만들기 위함이다.
    labels_ = tf.one_hot(labels, params['vocabulary_length'])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))

    accuracy = tf.metrics.accuracy(labels=labels, predictions=predict)

    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    assert TRAIN

    # lrate = d−0.5 *  model · min(step_num−0.5, step_num · warmup_steps−1.5)
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


# main.py

In [34]:
#import tensorflow as tf
#import model as ml
#import data
import numpy as np
import os
import sys
#from configs import DEFINES

In [35]:
DATA_OUT_PATH = './data_out/'
data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)
os.makedirs(data_out_path, exist_ok=True)

In [36]:
# 데이터를 통한 사전 구성 한다.
char2idx, idx2char, vocabulary_length = load_vocabulary() # data.load_vocabulary()

In [38]:
# 훈련 데이터와 테스트 데이터를 가져온다.
train_input, train_label, eval_input, eval_label = load_data() #data.load_data()

In [39]:
# 훈련셋 인코딩 만드는 부분이다.
train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx) # data.enc_processing
# 훈련셋 디코딩 입력 부분 만드는 부분이다.
train_output_dec, train_output_dec_length = dec_output_processing(train_label, char2idx) #data.dec_output_processing
# 훈련셋 디코딩 출력 부분 만드는 부분이다.
train_target_dec = dec_target_processing(train_label, char2idx) #data.dec_target_processing

In [40]:
# 평가셋 인코딩 만드는 부분이다.
eval_input_enc, eval_input_enc_length = enc_processing(eval_input, char2idx) #data.enc_processing()
# 평가셋 인코딩 만드는 부분이다.
eval_output_dec, eval_output_dec_length = dec_output_processing(eval_label, char2idx) #data.dec_output_processing()
# 평가셋 인코딩 만드는 부분이다.
eval_target_dec = dec_target_processing(eval_label, char2idx) #data.dec_target_processing()

In [41]:
# 현재 경로'./'에 현재 경로 하부에 
# 체크 포인트를 저장한 디렉토리를 설정한다.
check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
# 디렉토리를 만드는 함수이며 두번째 인자 exist_ok가 
# True이면 디렉토리가 이미 존재해도 OSError가 
# 발생하지 않는다.
# exist_ok가 False이면 이미 존재하면 
# OSError가 발생한다.
os.makedirs(check_point_path, exist_ok=True)

In [42]:
# 에스티메이터 구성한다.
classifier = tf.estimator.Estimator(
    model_fn= Model,  # 모델 등록한다. #ml.Model,
    model_dir=DEFINES.check_point_path,  # 체크포인트 위치 등록한다.
    params={  # 모델 쪽으로 파라메터 전달한다.
        'embedding_size': DEFINES.embedding_size,
        'model_hidden_size': DEFINES.model_hidden_size,  # 가중치 크기 설정한다.
        'ffn_hidden_size': DEFINES.ffn_hidden_size,
        'attention_head_size': DEFINES.attention_head_size,
        'learning_rate': DEFINES.learning_rate,  # 학습율 설정한다.
        'vocabulary_length': vocabulary_length,  # 딕셔너리 크기를 설정한다.
        'embedding_size': DEFINES.embedding_size,  # 임베딩 크기를 설정한다.
        'layer_size': DEFINES.layer_size,
        'max_sequence_length': DEFINES.max_sequence_length,
        'xavier_initializer': DEFINES.xavier_initializer
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a36248f10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
# 학습 실행
# data.train_input_fn()
classifier.train(input_fn=lambda: train_input_fn(
    train_input_enc, train_output_dec, train_target_dec, DEFINES.batch_size), steps=DEFINES.train_steps)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:loss = 9.851012, step = 1
INFO:tensorflow:global_step/sec: 0.374043
INFO:tensorflow:loss = 1.4228973, step = 101 (267.369 sec)
INFO:tensorflow:global_step/sec: 0.314627
INFO:tensorflow:loss = 1.201162, step = 201 (317.821 sec)
INFO:tensorflow:Saving checkpoints for 203 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_s

INFO:tensorflow:loss = 0.003938982, step = 5201 (189.230 sec)
INFO:tensorflow:Saving checkpoints for 5243 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.525703
INFO:tensorflow:loss = 0.0028721658, step = 5301 (190.221 sec)
INFO:tensorflow:global_step/sec: 0.528802
INFO:tensorflow:loss = 0.0032170797, step = 5401 (189.107 sec)
INFO:tensorflow:global_step/sec: 0.533101
INFO:tensorflow:loss = 0.0039913035, step = 5501 (187.581 sec)
INFO:tensorflow:Saving checkpoints for 5561 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.524473
INFO:tensorflow:loss = 0.002536189, step = 5601 (190.668 sec)
INFO:tensorflow:global_step/sec: 0.527866
INFO:tensorflow:loss = 0.0025542467, step = 5701 (189.442 sec)
INFO:tensorflow:global_step/sec: 0.526426
INFO:tensorflow:loss = 0.0026845068, step = 5801 (189.961 sec)
INFO:tensorflow:Saving checkpoints for 5877 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.520804
INFO:tensorflow

INFO:tensorflow:global_step/sec: 0.5389
INFO:tensorflow:loss = 0.0016833135, step = 11501 (185.564 sec)
INFO:tensorflow:Saving checkpoints for 11575 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.530334
INFO:tensorflow:loss = 0.0009750866, step = 11601 (188.560 sec)
INFO:tensorflow:global_step/sec: 0.536163
INFO:tensorflow:loss = 0.0027192459, step = 11701 (186.510 sec)
INFO:tensorflow:global_step/sec: 0.540752
INFO:tensorflow:loss = 0.00019574503, step = 11801 (184.929 sec)
INFO:tensorflow:Saving checkpoints for 11897 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.531678
INFO:tensorflow:loss = 0.000175193, step = 11901 (188.082 sec)
INFO:tensorflow:global_step/sec: 0.536698
INFO:tensorflow:loss = 0.00027460014, step = 12001 (186.326 sec)
INFO:tensorflow:global_step/sec: 0.538213
INFO:tensorflow:loss = 0.0022367402, step = 12101 (185.799 sec)
INFO:tensorflow:global_step/sec: 0.536107
INFO:tensorflow:loss = 0.00022231892, step = 12

INFO:tensorflow:global_step/sec: 0.994317
INFO:tensorflow:loss = 0.0024831328, step = 18201 (100.570 sec)
INFO:tensorflow:Saving checkpoints for 18284 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.991137
INFO:tensorflow:loss = 8.109043e-05, step = 18301 (100.894 sec)
INFO:tensorflow:global_step/sec: 0.996401
INFO:tensorflow:loss = 8.8777226e-05, step = 18401 (100.361 sec)
INFO:tensorflow:global_step/sec: 0.994689
INFO:tensorflow:loss = 6.7263376e-05, step = 18501 (100.535 sec)
INFO:tensorflow:global_step/sec: 0.994465
INFO:tensorflow:loss = 6.3202526e-05, step = 18601 (100.556 sec)
INFO:tensorflow:global_step/sec: 0.994895
INFO:tensorflow:loss = 0.0007166419, step = 18701 (100.513 sec)
INFO:tensorflow:global_step/sec: 0.992985
INFO:tensorflow:loss = 6.225588e-05, step = 18801 (100.707 sec)
INFO:tensorflow:Saving checkpoints for 18881 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.992595
INFO:tensorflow:loss = 0.001214084, step = 

In [46]:
"""
WARNING:tensorflow:From <ipython-input-26-9f687fe5de94>:23: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
WARNING:tensorflow:From /Users/csg/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:loss = 9.851012, step = 1
INFO:tensorflow:global_step/sec: 0.374043
INFO:tensorflow:loss = 1.4228973, step = 101 (267.369 sec)
INFO:tensorflow:global_step/sec: 0.314627
INFO:tensorflow:loss = 1.201162, step = 201 (317.821 sec)
INFO:tensorflow:Saving checkpoints for 203 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.30182
INFO:tensorflow:loss = 1.1092479, step = 301 (331.322 sec)
INFO:tensorflow:Saving checkpoints for 386 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.306266
INFO:tensorflow:loss = 1.2192594, step = 401 (326.512 sec)
INFO:tensorflow:global_step/sec: 0.316143
INFO:tensorflow:loss = 1.0574493, step = 501 (316.318 sec)
INFO:tensorflow:Saving checkpoints for 576 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.316095
INFO:tensorflow:loss = 1.2222314, step = 601 (316.357 sec)
INFO:tensorflow:global_step/sec: 0.402453
INFO:tensorflow:loss = 1.0814201, step = 701 (248.476 sec)
INFO:tensorflow:global_step/sec: 0.476895
INFO:tensorflow:loss = 0.92180926, step = 801 (209.700 sec)
INFO:tensorflow:Saving checkpoints for 831 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.483062
INFO:tensorflow:loss = 0.9952788, step = 901 (207.001 sec)
INFO:tensorflow:global_step/sec: 0.524489
INFO:tensorflow:loss = 0.71539414, step = 1001 (190.661 sec)
INFO:tensorflow:global_step/sec: 0.532379
INFO:tensorflow:loss = 0.7991382, step = 1101 (187.837 sec)
INFO:tensorflow:Saving checkpoints for 1142 into ./data_out/check_point/model.ckpt.
WARNING:tensorflow:From /Users/csg/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:global_step/sec: 0.529492
INFO:tensorflow:loss = 0.6781517, step = 1201 (188.861 sec)
INFO:tensorflow:global_step/sec: 0.534043
INFO:tensorflow:loss = 0.58390146, step = 1301 (187.251 sec)
INFO:tensorflow:global_step/sec: 0.535853
INFO:tensorflow:loss = 0.5347602, step = 1401 (186.618 sec)
INFO:tensorflow:Saving checkpoints for 1462 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.528781
INFO:tensorflow:loss = 0.45289192, step = 1501 (189.114 sec)
INFO:tensorflow:global_step/sec: 0.531792
INFO:tensorflow:loss = 0.46289173, step = 1601 (188.044 sec)
INFO:tensorflow:global_step/sec: 0.537708
INFO:tensorflow:loss = 0.50838, step = 1701 (185.975 sec)
INFO:tensorflow:Saving checkpoints for 1782 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.529718
INFO:tensorflow:loss = 0.45483956, step = 1801 (188.779 sec)
INFO:tensorflow:global_step/sec: 0.535236
INFO:tensorflow:loss = 0.4051933, step = 1901 (186.836 sec)
INFO:tensorflow:global_step/sec: 0.52658
INFO:tensorflow:loss = 0.41349488, step = 2001 (189.904 sec)
INFO:tensorflow:Saving checkpoints for 2101 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.527388
INFO:tensorflow:loss = 0.36510232, step = 2101 (189.621 sec)
INFO:tensorflow:global_step/sec: 0.534059
INFO:tensorflow:loss = 0.3750486, step = 2201 (187.237 sec)
INFO:tensorflow:global_step/sec: 0.531441
INFO:tensorflow:loss = 0.3399685, step = 2301 (188.169 sec)
INFO:tensorflow:global_step/sec: 0.533749
INFO:tensorflow:loss = 0.30457175, step = 2401 (187.354 sec)
INFO:tensorflow:Saving checkpoints for 2420 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.52908
INFO:tensorflow:loss = 0.22634828, step = 2501 (189.006 sec)
INFO:tensorflow:global_step/sec: 0.52111
INFO:tensorflow:loss = 0.22076538, step = 2601 (191.900 sec)
INFO:tensorflow:global_step/sec: 0.52975
INFO:tensorflow:loss = 0.2208243, step = 2701 (188.767 sec)
INFO:tensorflow:Saving checkpoints for 2736 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.522246
INFO:tensorflow:loss = 0.18072264, step = 2801 (191.482 sec)
INFO:tensorflow:global_step/sec: 0.528011
INFO:tensorflow:loss = 0.14577532, step = 2901 (189.388 sec)
INFO:tensorflow:global_step/sec: 0.529897
INFO:tensorflow:loss = 0.11771963, step = 3001 (188.717 sec)
INFO:tensorflow:Saving checkpoints for 3053 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.523991
INFO:tensorflow:loss = 0.08644039, step = 3101 (190.843 sec)
INFO:tensorflow:global_step/sec: 0.530104
INFO:tensorflow:loss = 0.06595415, step = 3201 (188.641 sec)
INFO:tensorflow:global_step/sec: 0.527708
INFO:tensorflow:loss = 0.07109492, step = 3301 (189.500 sec)
INFO:tensorflow:Saving checkpoints for 3370 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.520642
INFO:tensorflow:loss = 0.05894123, step = 3401 (192.068 sec)
INFO:tensorflow:global_step/sec: 0.530553
INFO:tensorflow:loss = 0.040457536, step = 3501 (188.484 sec)
INFO:tensorflow:global_step/sec: 0.510914
INFO:tensorflow:loss = 0.032512363, step = 3601 (195.727 sec)
INFO:tensorflow:Saving checkpoints for 3677 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.490508
INFO:tensorflow:loss = 0.025382118, step = 3701 (203.870 sec)
INFO:tensorflow:global_step/sec: 0.527103
INFO:tensorflow:loss = 0.029744934, step = 3801 (189.717 sec)
INFO:tensorflow:global_step/sec: 0.530721
INFO:tensorflow:loss = 0.0218961, step = 3901 (188.422 sec)
INFO:tensorflow:Saving checkpoints for 3994 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.522903
INFO:tensorflow:loss = 0.01825039, step = 4001 (191.239 sec)
INFO:tensorflow:global_step/sec: 0.530629
INFO:tensorflow:loss = 0.011400049, step = 4101 (188.456 sec)
INFO:tensorflow:global_step/sec: 0.53104
INFO:tensorflow:loss = 0.0079858, step = 4201 (188.311 sec)
INFO:tensorflow:global_step/sec: 0.529431
INFO:tensorflow:loss = 0.012822186, step = 4301 (188.883 sec)
INFO:tensorflow:Saving checkpoints for 4312 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.51841
INFO:tensorflow:loss = 0.010661416, step = 4401 (192.897 sec)
INFO:tensorflow:global_step/sec: 0.530194
INFO:tensorflow:loss = 0.008734199, step = 4501 (188.610 sec)
INFO:tensorflow:global_step/sec: 0.530084
INFO:tensorflow:loss = 0.006268938, step = 4601 (188.649 sec)
INFO:tensorflow:Saving checkpoints for 4628 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.460731
INFO:tensorflow:loss = 0.004935683, step = 4701 (217.047 sec)
INFO:tensorflow:global_step/sec: 0.506629
INFO:tensorflow:loss = 0.004445182, step = 4801 (197.382 sec)
INFO:tensorflow:global_step/sec: 0.527004
INFO:tensorflow:loss = 0.004622151, step = 4901 (189.752 sec)
INFO:tensorflow:Saving checkpoints for 4926 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.526097
INFO:tensorflow:loss = 0.0037921167, step = 5001 (190.079 sec)
INFO:tensorflow:global_step/sec: 0.528295
INFO:tensorflow:loss = 0.0035148915, step = 5101 (189.290 sec)
INFO:tensorflow:global_step/sec: 0.528454
INFO:tensorflow:loss = 0.003938982, step = 5201 (189.230 sec)
INFO:tensorflow:Saving checkpoints for 5243 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.525703
INFO:tensorflow:loss = 0.0028721658, step = 5301 (190.221 sec)
INFO:tensorflow:global_step/sec: 0.528802
INFO:tensorflow:loss = 0.0032170797, step = 5401 (189.107 sec)
INFO:tensorflow:global_step/sec: 0.533101
INFO:tensorflow:loss = 0.0039913035, step = 5501 (187.581 sec)
INFO:tensorflow:Saving checkpoints for 5561 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.524473
INFO:tensorflow:loss = 0.002536189, step = 5601 (190.668 sec)
INFO:tensorflow:global_step/sec: 0.527866
INFO:tensorflow:loss = 0.0025542467, step = 5701 (189.442 sec)
INFO:tensorflow:global_step/sec: 0.526426
INFO:tensorflow:loss = 0.0026845068, step = 5801 (189.961 sec)
INFO:tensorflow:Saving checkpoints for 5877 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.520804
INFO:tensorflow:loss = 0.05038212, step = 5901 (192.009 sec)
INFO:tensorflow:global_step/sec: 0.529231
INFO:tensorflow:loss = 0.0510709, step = 6001 (188.955 sec)
INFO:tensorflow:global_step/sec: 0.482363
INFO:tensorflow:loss = 0.048379704, step = 6101 (207.327 sec)
INFO:tensorflow:Saving checkpoints for 6176 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.476543
INFO:tensorflow:loss = 0.011431074, step = 6201 (209.829 sec)
INFO:tensorflow:global_step/sec: 0.451873
INFO:tensorflow:loss = 0.008037356, step = 6301 (221.316 sec)
INFO:tensorflow:global_step/sec: 0.492729
INFO:tensorflow:loss = 0.0022586512, step = 6401 (202.938 sec)
INFO:tensorflow:Saving checkpoints for 6460 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.494331
INFO:tensorflow:loss = 0.00212478, step = 6501 (202.293 sec)
INFO:tensorflow:global_step/sec: 0.534594
INFO:tensorflow:loss = 0.0027399966, step = 6601 (187.059 sec)
INFO:tensorflow:global_step/sec: 0.537273
INFO:tensorflow:loss = 0.0015368729, step = 6701 (186.127 sec)
INFO:tensorflow:Saving checkpoints for 6781 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.531341
INFO:tensorflow:loss = 0.0012392432, step = 6801 (188.201 sec)
INFO:tensorflow:global_step/sec: 0.53859
INFO:tensorflow:loss = 0.002306669, step = 6901 (185.671 sec)
INFO:tensorflow:global_step/sec: 0.538297
INFO:tensorflow:loss = 0.002583833, step = 7001 (185.770 sec)
INFO:tensorflow:global_step/sec: 0.537791
INFO:tensorflow:loss = 0.002243704, step = 7101 (185.946 sec)
INFO:tensorflow:Saving checkpoints for 7103 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.535682
INFO:tensorflow:loss = 0.00552486, step = 7201 (186.679 sec)
INFO:tensorflow:global_step/sec: 0.538337
INFO:tensorflow:loss = 0.0009523763, step = 7301 (185.755 sec)
INFO:tensorflow:global_step/sec: 0.540708
INFO:tensorflow:loss = 0.00075926527, step = 7401 (184.943 sec)
INFO:tensorflow:Saving checkpoints for 7427 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.534452
INFO:tensorflow:loss = 0.002653664, step = 7501 (187.107 sec)
INFO:tensorflow:global_step/sec: 0.536329
INFO:tensorflow:loss = 0.000850198, step = 7601 (186.455 sec)
INFO:tensorflow:global_step/sec: 0.532341
INFO:tensorflow:loss = 0.0007420227, step = 7701 (187.849 sec)
INFO:tensorflow:Saving checkpoints for 7748 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.534448
INFO:tensorflow:loss = 0.000739256, step = 7801 (187.108 sec)
INFO:tensorflow:global_step/sec: 0.537538
INFO:tensorflow:loss = 0.002246324, step = 7901 (186.034 sec)
INFO:tensorflow:global_step/sec: 0.540466
INFO:tensorflow:loss = 0.0015041614, step = 8001 (185.027 sec)
INFO:tensorflow:Saving checkpoints for 8071 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532736
INFO:tensorflow:loss = 0.0036575506, step = 8101 (187.708 sec)
INFO:tensorflow:global_step/sec: 0.537762
INFO:tensorflow:loss = 0.00055164867, step = 8201 (185.957 sec)
INFO:tensorflow:global_step/sec: 0.479131
INFO:tensorflow:loss = 0.0008920562, step = 8301 (208.713 sec)
INFO:tensorflow:Saving checkpoints for 8368 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.446693
INFO:tensorflow:loss = 0.00054696726, step = 8401 (223.864 sec)
INFO:tensorflow:global_step/sec: 0.534455
INFO:tensorflow:loss = 0.001171227, step = 8501 (187.108 sec)
INFO:tensorflow:global_step/sec: 0.537668
INFO:tensorflow:loss = 0.00058456906, step = 8601 (185.987 sec)
INFO:tensorflow:Saving checkpoints for 8682 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.530262
INFO:tensorflow:loss = 0.0011106952, step = 8701 (188.585 sec)
INFO:tensorflow:global_step/sec: 0.531993
INFO:tensorflow:loss = 0.0005706858, step = 8801 (187.975 sec)
INFO:tensorflow:global_step/sec: 0.534851
INFO:tensorflow:loss = 0.0004522126, step = 8901 (186.967 sec)
INFO:tensorflow:Saving checkpoints for 9001 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.523589
INFO:tensorflow:loss = 0.0022888358, step = 9001 (190.999 sec)
INFO:tensorflow:global_step/sec: 0.531804
INFO:tensorflow:loss = 0.00043055907, step = 9101 (188.031 sec)
INFO:tensorflow:global_step/sec: 0.533236
INFO:tensorflow:loss = 0.0017077121, step = 9201 (187.533 sec)
INFO:tensorflow:global_step/sec: 0.536893
INFO:tensorflow:loss = 0.0009985975, step = 9301 (186.257 sec)
INFO:tensorflow:Saving checkpoints for 9321 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.534068
INFO:tensorflow:loss = 0.0011137193, step = 9401 (187.242 sec)
INFO:tensorflow:global_step/sec: 0.537776
INFO:tensorflow:loss = 0.0018180235, step = 9501 (185.951 sec)
INFO:tensorflow:global_step/sec: 0.533814
INFO:tensorflow:loss = 0.038427316, step = 9601 (187.330 sec)
INFO:tensorflow:Saving checkpoints for 9643 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532979
INFO:tensorflow:loss = 0.039091762, step = 9701 (187.625 sec)
INFO:tensorflow:global_step/sec: 0.536975
INFO:tensorflow:loss = 0.024474612, step = 9801 (186.230 sec)
INFO:tensorflow:global_step/sec: 0.539448
INFO:tensorflow:loss = 0.010235513, step = 9901 (185.375 sec)
INFO:tensorflow:Saving checkpoints for 9965 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.533308
INFO:tensorflow:loss = 0.002503615, step = 10001 (187.507 sec)
INFO:tensorflow:global_step/sec: 0.535629
INFO:tensorflow:loss = 0.002485278, step = 10101 (186.698 sec)
INFO:tensorflow:global_step/sec: 0.540183
INFO:tensorflow:loss = 0.0017635636, step = 10201 (185.121 sec)
INFO:tensorflow:Saving checkpoints for 10287 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.530525
INFO:tensorflow:loss = 0.0015848522, step = 10301 (188.492 sec)
INFO:tensorflow:global_step/sec: 0.536599
INFO:tensorflow:loss = 0.00047133007, step = 10401 (186.359 sec)
INFO:tensorflow:global_step/sec: 0.536617
INFO:tensorflow:loss = 0.0012525964, step = 10501 (186.353 sec)
INFO:tensorflow:global_step/sec: 0.537191
INFO:tensorflow:loss = 0.0003859105, step = 10601 (186.155 sec)
INFO:tensorflow:Saving checkpoints for 10609 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.534615
INFO:tensorflow:loss = 0.00042911258, step = 10701 (187.050 sec)
INFO:tensorflow:global_step/sec: 0.537426
INFO:tensorflow:loss = 0.0029735405, step = 10801 (186.072 sec)
INFO:tensorflow:global_step/sec: 0.535686
INFO:tensorflow:loss = 0.00087567983, step = 10901 (186.677 sec)
INFO:tensorflow:Saving checkpoints for 10931 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532789
INFO:tensorflow:loss = 0.00041083316, step = 11001 (187.692 sec)
INFO:tensorflow:global_step/sec: 0.535939
INFO:tensorflow:loss = 0.00026243847, step = 11101 (186.588 sec)
INFO:tensorflow:global_step/sec: 0.537991
INFO:tensorflow:loss = 0.0020794803, step = 11201 (185.876 sec)
INFO:tensorflow:Saving checkpoints for 11253 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.533213
INFO:tensorflow:loss = 0.0015997058, step = 11301 (187.543 sec)
INFO:tensorflow:global_step/sec: 0.536347
INFO:tensorflow:loss = 0.001191943, step = 11401 (186.446 sec)
INFO:tensorflow:global_step/sec: 0.5389
INFO:tensorflow:loss = 0.0016833135, step = 11501 (185.564 sec)
INFO:tensorflow:Saving checkpoints for 11575 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.530334
INFO:tensorflow:loss = 0.0009750866, step = 11601 (188.560 sec)
INFO:tensorflow:global_step/sec: 0.536163
INFO:tensorflow:loss = 0.0027192459, step = 11701 (186.510 sec)
INFO:tensorflow:global_step/sec: 0.540752
INFO:tensorflow:loss = 0.00019574503, step = 11801 (184.929 sec)
INFO:tensorflow:Saving checkpoints for 11897 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.531678
INFO:tensorflow:loss = 0.000175193, step = 11901 (188.082 sec)
INFO:tensorflow:global_step/sec: 0.536698
INFO:tensorflow:loss = 0.00027460014, step = 12001 (186.326 sec)
INFO:tensorflow:global_step/sec: 0.538213
INFO:tensorflow:loss = 0.0022367402, step = 12101 (185.799 sec)
INFO:tensorflow:global_step/sec: 0.536107
INFO:tensorflow:loss = 0.00022231892, step = 12201 (186.532 sec)
INFO:tensorflow:Saving checkpoints for 12219 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532193
INFO:tensorflow:loss = 0.0003694683, step = 12301 (187.901 sec)
INFO:tensorflow:global_step/sec: 0.534682
INFO:tensorflow:loss = 0.00020233916, step = 12401 (187.027 sec)
INFO:tensorflow:global_step/sec: 0.535093
INFO:tensorflow:loss = 0.00016568856, step = 12501 (186.885 sec)
INFO:tensorflow:Saving checkpoints for 12540 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532076
INFO:tensorflow:loss = 0.0010975143, step = 12601 (187.942 sec)
INFO:tensorflow:global_step/sec: 0.535665
INFO:tensorflow:loss = 0.00016614042, step = 12701 (186.684 sec)
INFO:tensorflow:global_step/sec: 0.532986
INFO:tensorflow:loss = 0.00015966776, step = 12801 (187.622 sec)
INFO:tensorflow:Saving checkpoints for 12860 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.529012
INFO:tensorflow:loss = 0.00012263123, step = 12901 (189.032 sec)
INFO:tensorflow:global_step/sec: 0.537191
INFO:tensorflow:loss = 0.0011054545, step = 13001 (186.153 sec)
INFO:tensorflow:global_step/sec: 0.53977
INFO:tensorflow:loss = 0.0013276052, step = 13101 (185.265 sec)
INFO:tensorflow:Saving checkpoints for 13182 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.532313
INFO:tensorflow:loss = 0.000321368, step = 13201 (187.858 sec)
INFO:tensorflow:global_step/sec: 0.538891
INFO:tensorflow:loss = 0.000936568, step = 13301 (185.567 sec)
INFO:tensorflow:global_step/sec: 0.537483
INFO:tensorflow:loss = 0.0013770307, step = 13401 (186.068 sec)
INFO:tensorflow:global_step/sec: 0.684335
INFO:tensorflow:loss = 0.0001375194, step = 13501 (146.110 sec)
INFO:tensorflow:Saving checkpoints for 13548 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 1.01098
INFO:tensorflow:loss = 0.0017456012, step = 13601 (98.913 sec)
INFO:tensorflow:global_step/sec: 1.02371
INFO:tensorflow:loss = 0.00012119377, step = 13701 (97.684 sec)
INFO:tensorflow:global_step/sec: 1.0189
INFO:tensorflow:loss = 0.000110790366, step = 13801 (98.145 sec)
INFO:tensorflow:global_step/sec: 1.02057
INFO:tensorflow:loss = 0.00034207667, step = 13901 (97.986 sec)
INFO:tensorflow:global_step/sec: 1.02345
INFO:tensorflow:loss = 0.00011549039, step = 14001 (97.707 sec)
INFO:tensorflow:global_step/sec: 1.02233
INFO:tensorflow:loss = 0.0010131752, step = 14101 (97.816 sec)
INFO:tensorflow:Saving checkpoints for 14161 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 1.01186
INFO:tensorflow:loss = 0.0010847927, step = 14201 (98.827 sec)
INFO:tensorflow:global_step/sec: 1.02532
INFO:tensorflow:loss = 0.000600777, step = 14301 (97.532 sec)
INFO:tensorflow:global_step/sec: 1.01848
INFO:tensorflow:loss = 0.00026468822, step = 14401 (98.186 sec)
INFO:tensorflow:global_step/sec: 1.02453
INFO:tensorflow:loss = 0.00039609705, step = 14501 (97.605 sec)
INFO:tensorflow:global_step/sec: 1.02135
INFO:tensorflow:loss = 7.996263e-05, step = 14601 (97.909 sec)
INFO:tensorflow:global_step/sec: 0.993079
INFO:tensorflow:loss = 0.00010216706, step = 14701 (100.697 sec)
INFO:tensorflow:Saving checkpoints for 14771 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 1.00966
INFO:tensorflow:loss = 0.05573969, step = 14801 (99.044 sec)
INFO:tensorflow:global_step/sec: 1.01975
INFO:tensorflow:loss = 0.019230174, step = 14901 (98.063 sec)
INFO:tensorflow:global_step/sec: 1.01819
INFO:tensorflow:loss = 0.014932749, step = 15001 (98.214 sec)
INFO:tensorflow:global_step/sec: 1.02022
INFO:tensorflow:loss = 0.004116159, step = 15101 (98.018 sec)
INFO:tensorflow:global_step/sec: 1.02218
INFO:tensorflow:loss = 0.0057046693, step = 15201 (97.830 sec)
INFO:tensorflow:global_step/sec: 1.01163
INFO:tensorflow:loss = 0.0028741125, step = 15301 (98.851 sec)
INFO:tensorflow:Saving checkpoints for 15382 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 1.01352
INFO:tensorflow:loss = 0.00032028183, step = 15401 (98.666 sec)
INFO:tensorflow:global_step/sec: 1.02501
INFO:tensorflow:loss = 0.00023959711, step = 15501 (97.562 sec)
INFO:tensorflow:global_step/sec: 1.01907
INFO:tensorflow:loss = 0.00025196452, step = 15601 (98.126 sec)
INFO:tensorflow:global_step/sec: 1.02146
INFO:tensorflow:loss = 0.00024521482, step = 15701 (97.899 sec)
INFO:tensorflow:global_step/sec: 1.02202
INFO:tensorflow:loss = 0.00024659783, step = 15801 (97.845 sec)
INFO:tensorflow:global_step/sec: 1.01465
INFO:tensorflow:loss = 0.0007065743, step = 15901 (98.557 sec)
INFO:tensorflow:Saving checkpoints for 15994 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 1.01287
INFO:tensorflow:loss = 0.00056443224, step = 16001 (98.730 sec)
INFO:tensorflow:global_step/sec: 1.00246
INFO:tensorflow:loss = 0.0011978903, step = 16101 (99.754 sec)
INFO:tensorflow:global_step/sec: 0.949489
INFO:tensorflow:loss = 0.0011037946, step = 16201 (105.319 sec)
INFO:tensorflow:global_step/sec: 1.01088
INFO:tensorflow:loss = 0.0011561932, step = 16301 (98.926 sec)
INFO:tensorflow:global_step/sec: 1.01376
INFO:tensorflow:loss = 0.00071384193, step = 16401 (98.641 sec)
INFO:tensorflow:global_step/sec: 1.00338
INFO:tensorflow:loss = 0.001300743, step = 16501 (99.665 sec)
INFO:tensorflow:Saving checkpoints for 16585 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.936179
INFO:tensorflow:loss = 0.00014610842, step = 16601 (106.816 sec)
INFO:tensorflow:global_step/sec: 1.02159
INFO:tensorflow:loss = 0.0032634903, step = 16701 (97.888 sec)
INFO:tensorflow:global_step/sec: 1.00224
INFO:tensorflow:loss = 0.00015724216, step = 16801 (99.775 sec)
INFO:tensorflow:global_step/sec: 0.847253
INFO:tensorflow:loss = 0.00028053529, step = 16901 (118.031 sec)
INFO:tensorflow:global_step/sec: 0.695631
INFO:tensorflow:loss = 0.0001304517, step = 17001 (143.752 sec)
INFO:tensorflow:global_step/sec: 0.833393
INFO:tensorflow:loss = 0.00011266206, step = 17101 (119.991 sec)
INFO:tensorflow:Saving checkpoints for 17105 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.861444
INFO:tensorflow:loss = 0.00014104795, step = 17201 (116.084 sec)
INFO:tensorflow:global_step/sec: 0.994052
INFO:tensorflow:loss = 0.000102931335, step = 17301 (100.601 sec)
INFO:tensorflow:global_step/sec: 0.993935
INFO:tensorflow:loss = 0.0014931997, step = 17401 (100.608 sec)
INFO:tensorflow:global_step/sec: 0.995002
INFO:tensorflow:loss = 0.00037112282, step = 17501 (100.502 sec)
INFO:tensorflow:global_step/sec: 0.991886
INFO:tensorflow:loss = 0.001362523, step = 17601 (100.819 sec)
INFO:tensorflow:Saving checkpoints for 17687 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.98369
INFO:tensorflow:loss = 0.0027767164, step = 17701 (101.657 sec)
INFO:tensorflow:global_step/sec: 0.996227
INFO:tensorflow:loss = 0.00086015766, step = 17801 (100.379 sec)
INFO:tensorflow:global_step/sec: 0.991908
INFO:tensorflow:loss = 0.00010238517, step = 17901 (100.816 sec)
INFO:tensorflow:global_step/sec: 0.998
INFO:tensorflow:loss = 0.0019523215, step = 18001 (100.200 sec)
INFO:tensorflow:global_step/sec: 0.994563
INFO:tensorflow:loss = 7.101534e-05, step = 18101 (100.548 sec)
INFO:tensorflow:global_step/sec: 0.994317
INFO:tensorflow:loss = 0.0024831328, step = 18201 (100.570 sec)
INFO:tensorflow:Saving checkpoints for 18284 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.991137
INFO:tensorflow:loss = 8.109043e-05, step = 18301 (100.894 sec)
INFO:tensorflow:global_step/sec: 0.996401
INFO:tensorflow:loss = 8.8777226e-05, step = 18401 (100.361 sec)
INFO:tensorflow:global_step/sec: 0.994689
INFO:tensorflow:loss = 6.7263376e-05, step = 18501 (100.535 sec)
INFO:tensorflow:global_step/sec: 0.994465
INFO:tensorflow:loss = 6.3202526e-05, step = 18601 (100.556 sec)
INFO:tensorflow:global_step/sec: 0.994895
INFO:tensorflow:loss = 0.0007166419, step = 18701 (100.513 sec)
INFO:tensorflow:global_step/sec: 0.992985
INFO:tensorflow:loss = 6.225588e-05, step = 18801 (100.707 sec)
INFO:tensorflow:Saving checkpoints for 18881 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.992595
INFO:tensorflow:loss = 0.001214084, step = 18901 (100.746 sec)
INFO:tensorflow:global_step/sec: 0.999699
INFO:tensorflow:loss = 0.0011079323, step = 19001 (100.032 sec)
INFO:tensorflow:global_step/sec: 0.991756
INFO:tensorflow:loss = 6.469381e-05, step = 19101 (100.829 sec)
INFO:tensorflow:global_step/sec: 0.998561
INFO:tensorflow:loss = 6.493602e-05, step = 19201 (100.145 sec)
INFO:tensorflow:global_step/sec: 0.998909
INFO:tensorflow:loss = 0.0015367422, step = 19301 (100.109 sec)
INFO:tensorflow:global_step/sec: 0.993946
INFO:tensorflow:loss = 0.00020893176, step = 19401 (100.610 sec)
INFO:tensorflow:Saving checkpoints for 19479 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:global_step/sec: 0.986981
INFO:tensorflow:loss = 7.549991e-05, step = 19501 (101.318 sec)
INFO:tensorflow:global_step/sec: 0.805648
INFO:tensorflow:loss = 4.0136143e-05, step = 19601 (124.126 sec)
INFO:tensorflow:global_step/sec: 0.706749
INFO:tensorflow:loss = 0.0010814742, step = 19701 (141.494 sec)
INFO:tensorflow:global_step/sec: 0.627158
INFO:tensorflow:loss = 7.024543e-05, step = 19801 (159.447 sec)
INFO:tensorflow:global_step/sec: 0.739415
INFO:tensorflow:loss = 0.063375644, step = 19901 (135.242 sec)
INFO:tensorflow:Saving checkpoints for 19914 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:Saving checkpoints for 20000 into ./data_out/check_point/model.ckpt.
INFO:tensorflow:Loss for final step: 0.042772125.
<tensorflow_estimator.python.estimator.estimator.Estimator at 0x63401b450>
"""

'\nWARNING:tensorflow:From <ipython-input-26-9f687fe5de94>:23: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.\nInstructions for updating:\nUse `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.\nINFO:tensorflow:Calling model_fn.\nWARNING:tensorflow:From /Users/csg/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.\nInstructions for updating:\nIf using Keras pass *_constraint arguments to layers.\nINFO:tensorflow:Done calling model_fn.\nINFO:tensorflow:Create CheckpointSaverHook.\nINFO:tensorflow:Graph was finalized.\nINFO:tensorflow:

In [47]:
# data.eval_input_fn()
eval_result = classifier.evaluate(input_fn=lambda: eval_input_fn(
    eval_input_enc, eval_output_dec, eval_target_dec, DEFINES.batch_size))
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-07T22:51:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-09-07-22:52:33
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.8880729, global_step = 20000, loss = 1.3867551
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ./data_out/check_point/model.ckpt-20000

EVAL set accuracy: 0.888



In [48]:
"""
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-07T22:51:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-09-07-22:52:33
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.8880729, global_step = 20000, loss = 1.3867551
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ./data_out/check_point/model.ckpt-20000

EVAL set accuracy: 0.888
"""

"\nINFO:tensorflow:Calling model_fn.\nINFO:tensorflow:Done calling model_fn.\nINFO:tensorflow:Starting evaluation at 2020-09-07T22:51:48Z\nINFO:tensorflow:Graph was finalized.\nINFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000\nINFO:tensorflow:Running local_init_op.\nINFO:tensorflow:Done running local_init_op.\nINFO:tensorflow:Finished evaluation at 2020-09-07-22:52:33\nINFO:tensorflow:Saving dict for global step 20000: accuracy = 0.8880729, global_step = 20000, loss = 1.3867551\nINFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ./data_out/check_point/model.ckpt-20000\n\nEVAL set accuracy: 0.888\n"

In [49]:
# 테스트용 데이터 만드는 부분이다.
# 인코딩 부분 만든다.
predic_input_enc, predic_input_enc_length = enc_processing(["가끔 궁금해"], char2idx) #data.enc_processing()
# 학습 과정이 아니므로 디코딩 입력은 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_output_dec, predic_output_decLength = dec_output_processing([""], char2idx) #data.dec_output_processing()
# 학습 과정이 아니므로 디코딩 출력 부분도 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_target_dec = dec_target_processing([""], char2idx) #data.dec_target_processing

In [53]:
for i in range(DEFINES.max_sequence_length):
    print(i)
    if i > 0:
        predic_output_dec, predic_output_decLength = dec_output_processing([answer], char2idx) #data.dec_output_processing()
        predic_target_dec = dec_target_processing([answer], char2idx)# data.dec_target_processing()
    
    # 예측을 하는 부분이다.
    # data.eval_input_fn()
    predictions = classifier.predict(
        input_fn=lambda: eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, 1))

    # 예측한 값을 인지 할 수 있도록
    # 텍스트로 변경하는 부분이다.
    # data.pred_next_string()
    answer, finished = pred_next_string(predictions, idx2char)
    print("{}: {}".format(i, answer))

    if finished:
        break

print("answer: ", answer)

0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0: 그 그렇게 되실 거예요 
answer:  그 그렇게 되실 거예요 


In [ ]:
"""
0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0: 그 그렇게 되실 거예요 
answer:  그 그렇게 되실 거예요 
"""

# predict.py

In [8]:
"""
#import tensorflow as tf
#import data
import os
import sys
#import model as ml

#from configs import DEFINES

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.ERROR)
    arg_length = len(sys.argv)

    if (arg_length < 2):
        raise Exception("Don't call us. We'll call you")

    # 데이터를 통한 사전 구성 한다.
    char2idx, idx2char, vocabulary_length = data.load_vocabulary()

    # 테스트용 데이터 만드는 부분이다.
    # 인코딩 부분 만든다.
    input = ""
    for i in sys.argv[1:]:
        input += i
        input += " "

    print(input)
    predic_input_enc, predic_input_enc_length = data.enc_processing([input], char2idx)
    # 학습 과정이 아니므로 디코딩 입력은 
    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
    predic_output_dec, predic_output_dec_length = data.dec_output_processing([""], char2idx)
    # 학습 과정이 아니므로 디코딩 출력 부분도 
    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
    predic_target_dec = data.dec_target_processing([""], char2idx)

    # 에스티메이터 구성한다.
    classifier = tf.estimator.Estimator(
        model_fn=ml.Model,  # 모델 등록한다.
        model_dir=DEFINES.check_point_path,  # 체크포인트 위치 등록한다.
        params={  # 모델 쪽으로 파라메터 전달한다.
            'embedding_size': DEFINES.embedding_size,
            'model_hidden_size': DEFINES.model_hidden_size,  # 가중치 크기 설정한다.
            'ffn_hidden_size': DEFINES.ffn_hidden_size,
            'attention_head_size': DEFINES.attention_head_size,
            'learning_rate': DEFINES.learning_rate,  # 학습율 설정한다.
            'vocabulary_length': vocabulary_length,  # 딕셔너리 크기를 설정한다.
            'embedding_size': DEFINES.embedding_size,  # 임베딩 크기를 설정한다.
            'layer_size': DEFINES.layer_size,
            'max_sequence_length': DEFINES.max_sequence_length,
            'xavier_initializer': DEFINES.xavier_initializer
        })

    for i in range(DEFINES.max_sequence_length):
        if i > 0:
            predic_output_dec, predic_output_decLength = data.dec_output_processing([answer], char2idx)
            predic_target_dec = data.dec_target_processing([answer], char2idx)
        # 예측을 하는 부분이다.
        predictions = classifier.predict(
            input_fn=lambda: data.eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, 1))

        answer, finished = data.pred_next_string(predictions, idx2char)

        if finished:
            break

    # 예측한 값을 인지 할 수 있도록
    # 텍스트로 변경하는 부분이다.
    print("answer: ", answer)
"""

'\n#import tensorflow as tf\n#import data\nimport os\nimport sys\n#import model as ml\n\n#from configs import DEFINES\n\nos.environ[\'TF_CPP_MIN_LOG_LEVEL\'] = \'3\'\n\n\nif __name__ == \'__main__\':\n    tf.logging.set_verbosity(tf.logging.ERROR)\n    arg_length = len(sys.argv)\n\n    if (arg_length < 2):\n        raise Exception("Don\'t call us. We\'ll call you")\n\n    # 데이터를 통한 사전 구성 한다.\n    char2idx, idx2char, vocabulary_length = data.load_vocabulary()\n\n    # 테스트용 데이터 만드는 부분이다.\n    # 인코딩 부분 만든다.\n    input = ""\n    for i in sys.argv[1:]:\n        input += i\n        input += " "\n\n    print(input)\n    predic_input_enc, predic_input_enc_length = data.enc_processing([input], char2idx)\n    # 학습 과정이 아니므로 디코딩 입력은 \n    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)\n    predic_output_dec, predic_output_dec_length = data.dec_output_processing([""], char2idx)\n    # 학습 과정이 아니므로 디코딩 출력 부분도 \n    # 존재하지 않는다.(구조를 맞추기 위해 넣는다.)\n    predic_target_dec = data.dec_target_processing([""], char2idx)\n\n    